In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

os.environ.update({'OPENAI_API_KEY': openai_api_key})

### chain 설정

In [3]:
from langchain.prompts import load_prompt, PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.embeddings import OpenAIEmbeddings

In [22]:
template = '''
I want you to generate search query to find evidence for writing an article.
I will give you the table of contents with purpose of the article, and I will need you to generate search query in keyword.
Each keyword is used to retrieve data from the internet.
Only say the keyword queries. But if there is none, say nothing, and don't give me an explanation.
Generate no more than three keyword queries.
Answer in Korean.

purpose for the article:
===
{purpose}
===

table of contents:
===
{table}
===

keyword quries:
'''

In [23]:
class KeywordsChain:
    def __init__(self, keywords_template=None, keywords_template_path='../openai_skt/models/templates/keywords_prompt.json', verbose=False) -> None:
        if keywords_template is not None:
            self.keywords_prompt = PromptTemplate(
                    input_variables=["purpose","table"],
                    template=keywords_template,
                )
        else:
            self.keywords_template_path = keywords_template_path
            self.keywords_prompt = load_prompt(self.keywords_template_path)
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
        self.verbose = verbose
        self.keywords_chain = LLMChain(llm=self.llm, prompt=self.keywords_prompt, verbose=self.verbose)

    def run(self, purpose:str=None, table:str=None):
        input_dict = {'purpose': purpose, 'table' : table}
        keywords = self.keywords_chain.run(input_dict)
        return keywords

    async def arun(self, purpose:str=None):
        input_dict = {'purpose': purpose}
        keywords = await self.keywords_chain.arun(input_dict)
        return keywords

In [24]:
keywords_chain = KeywordsChain(keywords_template=template, verbose=True)

In [25]:
table_contents = '''1. 서론
- 축구의 세계적인 인기와 영향력
2. 역사적 배경
- 축구의 기원과 발전
- 국제 축구 기구의 역할
3. 경기의 매력
- 스포츠의 엔터테인먼트 요소
- 다양한 팀과 선수들의 경기력
4. 문화적 영향
- 축구와 국가 간의 연결고리
- 축구 문화의 전파와 영향력
5. 경제적 영향
- 축구 산업의 성장과 경제적 이익
- 축구 관련 비즈니스와 스폰서십
6. 사회적 영향
- 축구의 사회적 통합과 다양성 증진
- 축구를 통한 사회적 문제 해결
7. 결론
- 축구의 세계적인 인기와 그 이유
- 개인적인 경험과 생각
'''

In [26]:
result = keywords_chain.run(purpose="축구가 세계적으로 인기있는 스포츠인 이유", table=table_contents)



> Entering new LLMChain chain...
Prompt after formatting:

I want you to generate search query to find evidence for writing an article.
I will give you the table of contents with purpose of the article, and I will need you to generate search query in keyword.
Each keyword is used to retrieve data from the internet.
Only say the keyword queries. But if there is none, say nothing, and don't give me an explanation.
Generate no more than three keyword queries.
Answer in Korean.

purpose for the article:
===
축구가 세계적으로 인기있는 스포츠인 이유
===

table of contents:
===
1. 서론
- 축구의 세계적인 인기와 영향력
2. 역사적 배경
- 축구의 기원과 발전
- 국제 축구 기구의 역할
3. 경기의 매력
- 스포츠의 엔터테인먼트 요소
- 다양한 팀과 선수들의 경기력
4. 문화적 영향
- 축구와 국가 간의 연결고리
- 축구 문화의 전파와 영향력
5. 경제적 영향
- 축구 산업의 성장과 경제적 이익
- 축구 관련 비즈니스와 스폰서십
6. 사회적 영향
- 축구의 사회적 통합과 다양성 증진
- 축구를 통한 사회적 문제 해결
7. 결론
- 축구의 세계적인 인기와 그 이유
- 개인적인 경험과 생각

===

keyword quries:


> Finished chain.


In [27]:
print(result)

축구 세계적 인기
축구 역사
축구 경제적 영향
